#Source Code

##Write a program in Pythonwhich solves the program without any parallelization.

In [1]:
import numpy as np

class PiCalculator:
    def __init__(self, N):
        self.N = N  # Number of rectangles

    def compute_pi(self):
        delta_x = 1 / self.N  # Width of each small rectangle
        total_area = 0  # Accumulator for the sum of areas of rectangles

        # Loop to calculate area of each rectangle
        for i in range(self.N):
            x_i = i * delta_x  # x-coordinate at the left side of the rectangle
            f_x = np.sqrt(1 - x_i**2)  # Height of the rectangle
            total_area += delta_x * f_x  # Add area of the rectangle to total

        # Multiply by 4 to get the area of the whole circle
        pi_approximation = 4 * total_area
        return pi_approximation

# Example usage:
calculator = PiCalculator(N=100000)
pi_approx1 = calculator.compute_pi()
print(f"Approximation of π with N={calculator.N}: {pi_approx1}")


Approximation of π with N=100000: 3.1416126164019564


##Write a program in Python which usesparallel computing via multiprocessingto solve the problem.

In [2]:
import numpy as np
from multiprocessing import Pool

def rectangle_area(x):
    """Compute the area of a single rectangle for the given x value."""
    delta_x = 1 / N  # Width of each rectangle, defined globally
    f_x = np.sqrt(1 - x**2)  # Height of the rectangle
    return delta_x * f_x  # Area of the rectangle

def compute_pi_multiprocessing(N, num_processes=None):
    """Compute pi using numerical integration with multiprocessing."""
    delta_x = 1 / N
    x_values = [i * delta_x for i in range(N)]  # x-coordinates of the left side of each rectangle

    with Pool(processes=num_processes) as pool:
        areas = pool.map(rectangle_area, x_values)  # Parallel computation of rectangle areas

    total_area = sum(areas)  # Summing up the areas calculated by different processes
    return 4 * total_area  # Multiplying by 4 to get the approximation of pi for the entire circle

# Example usage:
N = 100000  # Number of rectangles
num_processes = 4  # Number of processes to use (adjust based on your CPU)
pi_approx2 = compute_pi_multiprocessing(N, num_processes)
print(f"Approximation of π with N={N} and multiprocessing: {pi_approx2}")


Approximation of π with N=100000 and multiprocessing: 3.1416126164019564


##Write a program in Python which uses distributed parallel computing via mi4pyto solve the problem.

In [3]:
%pip install mpi4py

In [4]:
!sudo apt-get install -y python3-mpi4py

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-mpi4py is already the newest version (3.1.3-1build2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [5]:
!apt-get install -qq mpich
!pip install mpi4py

In [6]:
from mpi4py import MPI
import numpy as np

def compute_pi_mpi(N):
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()  # Get the rank of the current process
    size = comm.Get_size()  # Get the total number of processes

    delta_x = 1.0 / N
    local_n = N // size  # Divide the task equally among processes

    # Calculate the interval of integration for each process
    local_a = rank * local_n * delta_x
    local_b = local_a + local_n * delta_x

    local_sum = 0.0
    for i in range(local_n):
        x = local_a + (i + 0.5) * delta_x
        local_sum += np.sqrt(1 - x**2) * delta_x

    # Use MPI to collect all local integrals at the root process
    total_sum = comm.reduce(local_sum, op=MPI.SUM, root=0)

    if rank == 0:
        pi_approx = 4 * total_sum
        return pi_approx

# Example usage
if __name__ == "__main__":
    N = 100000  # Number of rectangles
    pi_approx3 = compute_pi_mpi(N)
    if pi_approx3 is not None:
        print(f"Approximation of π with N={N} using MPI: {pi_approx3}")


Approximation of π with N=100000 using MPI: 3.1415926644818337


#Profiling

##Profiling Nonparallel

In [27]:
import numpy as np
import cProfile

class PiCalculator:
    def __init__(self, N):
        self.N = N  # Number of rectangles

    def compute_pi(self):
        delta_x = 1 / self.N  # Width of each small rectangle
        total_area = 0  # Accumulator for the sum of areas of rectangles

        # Loop to calculate area of each rectangle
        for i in range(self.N):
            x_i = i * delta_x  # x-coordinate at the left side of the rectangle
            f_x = np.sqrt(1 - x_i**2)  # Height of the rectangle
            total_area += delta_x * f_x  # Add area of the rectangle to total

        # Multiply by 4 to get the area of the whole circle
        pi_approximation = 4 * total_area
        return pi_approximation

def profile_compute_pi():
    calculator = PiCalculator(N=100000)
    return calculator.compute_pi()

# Perform profiling
cProfile.run('profile_compute_pi()')


         6 function calls in 0.276 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.276    0.276 <ipython-input-27-4e0d15690064>:22(profile_compute_pi)
        1    0.000    0.000    0.000    0.000 <ipython-input-27-4e0d15690064>:5(__init__)
        1    0.276    0.276    0.276    0.276 <ipython-input-27-4e0d15690064>:8(compute_pi)
        1    0.000    0.000    0.276    0.276 <string>:1(<module>)
        1    0.000    0.000    0.276    0.276 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




##Profiling Multiprocessing

In [28]:
import numpy as np
from multiprocessing import Pool
import cProfile

class PiCalculatorMultiprocessing:
    def __init__(self, N, num_processes=None):
        self.N = N
        self.num_processes = num_processes
        self.delta_x = 1 / N  # Define delta_x here to avoid global dependency

    def rectangle_area(self, x):
        """Compute the area of a single rectangle for the given x value."""
        f_x = np.sqrt(1 - x**2)  # Height of the rectangle
        return self.delta_x * f_x  # Area of the rectangle

    def compute_pi(self):
        """Compute pi using numerical integration with multiprocessing."""
        x_values = [i * self.delta_x for i in range(self.N)]  # x-coordinates of the left side of each rectangle

        with Pool(processes=self.num_processes) as pool:
            areas = pool.map(self.rectangle_area, x_values)  # Parallel computation of rectangle areas

        total_area = sum(areas)  # Summing up the areas calculated by different processes
        return 4 * total_area  # Multiplying by 4 to get the approximation of pi for the entire circle

    def profile_compute_pi(self):
        """Function to profile the compute_pi method."""
        cProfile.runctx('self.compute_pi()', globals(), locals())


# Example usage:
calculator = PiCalculatorMultiprocessing(N=100000, num_processes=4)
pi_approx2 = calculator.compute_pi()
print(f"Approximation of π with N={calculator.N} and multiprocessing: {pi_approx2}")
# To profile the computation
calculator.profile_compute_pi()


Approximation of π with N=100000 and multiprocessing: 3.1416126164019564
         1177 function calls in 0.615 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       14    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:404(parent)
        1    0.000    0.000    0.613    0.613 <ipython-input-28-413c049606ea>:16(compute_pi)
        1    0.014    0.014    0.014    0.014 <ipython-input-28-413c049606ea>:18(<listcomp>)
        1    0.003    0.003    0.615    0.615 <string>:1(<module>)
        4    0.000    0.000    0.000    0.000 __init__.py:219(_acquireLock)
        4    0.000    0.000    0.000    0.000 __init__.py:228(_releaseLock)
        4    0.000    0.000    0.000    0.000 _weakrefset.py:39(_remove)
        7    0.000    0.000    0.000    0.000 _weakrefset.py:86(add)
        6    0.000    0.000    0.000    0.000 connection.py:117(__init__)
        6    0.000    0.000    0.000    0.000 connection.py:130(__del

#Profiling mpi4py

In [9]:
from mpi4py import MPI
import numpy as np
import cProfile

class PiCalculatorMPI:
    def __init__(self, N):
        self.N = N
        self.comm = MPI.COMM_WORLD
        self.rank = self.comm.Get_rank()
        self.size = self.comm.Get_size()
        self.delta_x = 1.0 / N

    def compute_pi(self):
        local_n = self.N // self.size  # Divide the task equally among processes

        # Calculate the interval of integration for each process
        local_a = self.rank * local_n * self.delta_x
        local_b = local_a + local_n * self.delta_x

        local_sum = 0.0
        for i in range(local_n):
            x = local_a + (i + 0.5) * self.delta_x
            local_sum += np.sqrt(1 - x**2) * self.delta_x

        # Use MPI to collect all local integrals at the root process
        total_sum = self.comm.reduce(local_sum, op=MPI.SUM, root=0)

        if self.rank == 0:
            pi_appro3x = 4 * total_sum
            return pi_approx3

    def profile_compute_pi(self):
        """Function to profile the compute_pi method using cProfile."""
        cProfile.runctx('self.compute_pi()', globals(), locals())

# Example usage
if __name__ == "__main__":
    calculator = PiCalculatorMPI(N=100000)
    if calculator.rank == 0:
        pi_approx3 = calculator.compute_pi()
        if pi_approx3 is not None:
            print(f"Approximation of π with N={calculator.N} using MPI: {pi_approx3}")

        # To profile the computation only from the root process
        calculator.profile_compute_pi()


Approximation of π with N=100000 using MPI: 3.1415926644818337
         6 function calls in 0.162 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.162    0.162    0.162    0.162 <ipython-input-9-d0784e20fbe8>:13(compute_pi)
        1    0.000    0.000    0.162    0.162 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 __init__.py:144(_DType_reduce)
        1    0.000    0.000    0.162    0.162 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'reduce' of 'mpi4py.MPI.Comm' objects}




#Result for the first code

In [10]:
pi_approx1

3.1416126164019564

#Result for the Second code

In [11]:
pi_approx2

3.1416126164019564

#Result for the Thrid Code

In [12]:
pi_approx3

3.1415926644818337